In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning

## Homework 8: Introduction to Computer vision and Time Series (Lectures 19 and 20) 

**Due date: see the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points:2}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset and storing it under the `data` folder. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture demo, we had different measurements for each Location. 

We want you to consider this for the avocado prices dataset. For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 4

Here, we can see that for different types of avocado (conventional and organic), we have two seperate observations that belong to a the same day and location

For example, on 2015-12-27 in Albany, conventional avocados had an average price of 1.33 but in the same region and and the same date we have organic avocados at 1.83.

In [9]:
df[df['type'] == 'conventional']

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.00,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.00,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.00,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.00,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.00,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2018-02-04,0.76,1272039.80,531469.08,368948.26,13078.20,358544.26,127833.03,229786.03,925.20,conventional,2018,WestTexNewMexico
8,2018-01-28,0.85,957086.16,479147.93,179489.19,7314.98,291134.06,143430.96,147376.43,326.67,conventional,2018,WestTexNewMexico
9,2018-01-21,0.84,1020913.20,505263.29,177911.40,9468.95,328269.56,118978.50,209131.06,160.00,conventional,2018,WestTexNewMexico
10,2018-01-14,0.90,950954.60,463945.73,188126.02,11227.47,287655.38,125408.69,162040.02,206.67,conventional,2018,WestTexNewMexico


In [10]:
df[df['type'] == 'organic']

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.83,989.55,8.16,88.59,0.00,892.80,892.80,0.00,0.0,organic,2015,Albany
1,2015-12-20,1.89,1163.03,30.24,172.14,0.00,960.65,960.65,0.00,0.0,organic,2015,Albany
2,2015-12-13,1.85,995.96,10.44,178.70,0.00,806.82,806.82,0.00,0.0,organic,2015,Albany
3,2015-12-06,1.84,1158.42,90.29,104.18,0.00,963.95,948.52,15.43,0.0,organic,2015,Albany
4,2015-11-29,1.94,831.69,0.00,94.73,0.00,736.96,736.96,0.00,0.0,organic,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
8,2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
9,2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
10,2018-01-14,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 4

We can see that the observations were taken every week in all locations except in WestTexNewMexico, where there are 3 less total observations and 1 instance of the space being 14 days and the other being 21 days

In [11]:
#adapted from lecture 20

def time_spacing_for_df(df):
    
    
    df['Date'] = pd.to_datetime(df['Date'])
    
    region = df['region'].unique()
    types = df['type'].unique()
    
    for region in region:
        for type in types:
            
            region_type_data = df[(df['region'] == region) & (df['type'] == type)]
            time_diffs = region_type_data['Date'].sort_values().diff().dropna().value_counts().sort_index()
        
            print(f"{region},{type},{time_diffs}")
            
time_spacing_for_df(df)

Albany,conventional,Date
7 days    168
Name: count, dtype: int64
Albany,organic,Date
7 days    168
Name: count, dtype: int64
Atlanta,conventional,Date
7 days    168
Name: count, dtype: int64
Atlanta,organic,Date
7 days    168
Name: count, dtype: int64
BaltimoreWashington,conventional,Date
7 days    168
Name: count, dtype: int64
BaltimoreWashington,organic,Date
7 days    168
Name: count, dtype: int64
Boise,conventional,Date
7 days    168
Name: count, dtype: int64
Boise,organic,Date
7 days    168
Name: count, dtype: int64
Boston,conventional,Date
7 days    168
Name: count, dtype: int64
Boston,organic,Date
7 days    168
Name: count, dtype: int64
BuffaloRochester,conventional,Date
7 days    168
Name: count, dtype: int64
BuffaloRochester,organic,Date
7 days    168
Name: count, dtype: int64
California,conventional,Date
7 days    168
Name: count, dtype: int64
California,organic,Date
7 days    168
Name: count, dtype: int64
Charlotte,conventional,Date
7 days    168
Name: count, dtype: int64
Cha

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 4

Most of the regions in the data set are citis for example Boston, Houston, and Seattle. However, among the regions there also "directions" that can cause overlap with the distinct cities, like Midsouth, West and North East. Also there are entire states like California and South Carolina, and also a Total US. So these can definitly cause some overlap

In [12]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2023W1/tree/main/lectures), with some improvements.

In [13]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [14]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [15]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [16]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 `AveragePrice` baseline 
rubric={points}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

the train r^2 is 0.8285800937261841


 the test r^2 is 0.7631780188583048

In [17]:

train_r2 = r2_score(df_train['AveragePriceNextWeek'], df_train['AveragePrice'])
test_r2 = r2_score(df_test['AveragePriceNextWeek'], df_test['AveragePrice'])

print(f"the train r^2 is {train_r2}")
print(f" the test r^2 is {test_r2}")

the train r^2 is 0.8285800937261841
 the test r^2 is 0.7631780188583048


In [18]:
assert not train_r2 is None, "Are you using the correct variable name?"
assert not test_r2 is None, "Are you using the correct variable name?"
assert sha1(str(round(train_r2, 3)).encode('utf8')).hexdigest() == 'b1136fe2a8918904393ab6f40bfb3f38eac5fc39', "Your training score is not correct. Are you using the right features?"
assert sha1(str(round(test_r2, 3)).encode('utf8')).hexdigest() == 'cc24d9a9b567b491a56b42f7adc582f2eefa5907', "Your test score is not correct. Are you using the right features?"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 10

### The R^2 score of the Basic date is 

Train R^2 Score: 0.9750190202543292


Test R^2 Score: 0.8458262956093439



### The R^2 score of the POSIX

Train R^2 Score: 0.9782232090941616

Test R^2 Score: 0.8481531762612489

For me, they performed very similarily, therefore it seems that a POSIX representation of time and a regular representation of the date are equally efficient at capturing the price patterns in this data set

##  1:Basic date 

In [19]:
from sklearn.model_selection import train_test_split


df_basic_date = df_hastarget.copy()

df_basic_date['Year'], df_basic_date['Month'], df_basic_date['Day'] = df_hastarget['Date'].dt.year, df_hastarget['Date'].dt.month, df_hastarget['Date'].dt.day

X_basic_date = df_basic_date.drop(columns=['AveragePriceNextWeek', "Date"])
y_basic_date = df_hastarget['AveragePriceNextWeek']




X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_basic_date, y_basic_date, test_size=0.3, random_state=0)

#to create numeric observations for type and region
X_train_b = pd.get_dummies(X_train_b, columns=['type', 'region'])
X_test_b = pd.get_dummies(X_test_b, columns=['type', 'region'])


In [20]:
y_train_b

9748     2.18
3372     1.59
9704     2.01
15956    0.85
3680     1.69
         ... 
9279     1.03
13201    1.07
9903     1.22
10863    1.18
2748     1.17
Name: AveragePriceNextWeek, Length: 12698, dtype: float64

In [21]:
X_train_b

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year,...,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
9748,1.86,147248.57,30270.92,85793.31,114.28,31070.06,30832.74,237.32,0.00,2017,...,False,False,False,False,False,False,False,False,False,False
3372,1.69,11196.30,628.58,4438.31,0.00,6129.41,4339.23,1790.18,0.00,2018,...,False,False,False,False,False,False,False,False,False,False
9704,2.02,34788.28,4328.47,15961.11,2631.05,11867.65,11026.93,840.72,0.00,2016,...,False,False,False,False,False,False,False,False,False,False
15956,0.84,127403.24,32460.64,34939.02,3173.84,56829.74,55178.16,1553.14,98.44,2016,...,False,False,False,True,False,False,False,False,False,False
3680,1.38,9167.98,584.28,2285.45,0.00,6298.25,1024.34,5273.91,0.00,2017,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9279,1.06,251601.16,144316.93,21840.90,1803.00,83640.33,71291.13,11513.64,835.56,2017,...,False,False,False,False,False,False,False,False,False,False
13201,1.08,135281.35,55106.27,46302.99,68.28,33803.81,32381.59,1422.22,0.00,2015,...,False,False,False,False,False,False,False,False,False,False
9903,1.32,3381321.05,102683.58,2390081.54,19968.52,868587.41,733966.40,132518.00,2103.01,2016,...,False,False,False,False,False,False,False,False,False,False
10863,1.23,275091.78,11335.64,195284.31,533.71,67938.12,62262.05,5676.07,0.00,2015,...,False,False,False,False,False,False,False,False,False,False


In [22]:



b_model = RandomForestRegressor(n_estimators=20, random_state=0)
b_model.fit(X_train_b, y_train_b)


print(f"Train R^2 Score: {r2_score(y_train_b, b_model.predict(X_train_b))}")
print(f"Test R^2 Score: {r2_score(y_test_b, b_model.predict(X_test_b))}")

Train R^2 Score: 0.9750190202543292
Test R^2 Score: 0.8458262956093439


In [23]:

feature_names = X_train_b.columns


data = {
    "Importance": b_model.feature_importances_,
}
pd.DataFrame(data=data, index=feature_names,).sort_values(
    by="Importance", ascending=False
)[:10]

,Importance
AveragePrice,0.839917
Small Bags,0.017936
4046,0.017705
Day,0.014567
4225,0.013422
Month,0.012634
Large Bags,0.012185
Total Bags,0.010342
Total Volume,0.009928
4770,0.008329


## POSIX time feature

In [24]:

df_posix_date = df_hastarget.copy()

# Convert to POSIX time
df_posix_date['POSIX_Time'] = df_hastarget['Date'].astype('int64') // 10 ** 9


X_posix_date = df_posix_date.drop(columns=['AveragePriceNextWeek', 'Date'])
y_posix_date = df_hastarget['AveragePriceNextWeek']


X_train_posix, X_test_posix, y_train_posix, y_test_posix = train_test_split(
    X_posix_date, y_posix_date, test_size=0.3, random_state=0
)


X_train_posix = pd.get_dummies(X_train_posix, columns=['type', 'region'])
X_test_posix = pd.get_dummies(X_test_posix, columns=['type', 'region'])


X_train_posix, X_test_posix = X_train_posix.align(X_test_posix, join='left', axis=1)
X_test_posix = X_test_posix.fillna(0) 

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train_posix, y_train_posix)


print(f"Train R^2 Score: {r2_score(y_train_posix, model.predict(X_train_posix))}")
print(f"Test R^2 Score: {r2_score(y_test_posix, model.predict(X_test_posix))}")

Train R^2 Score: 0.9782232090941616
Test R^2 Score: 0.8481531762612489


## Exercise 2: Short answer questions

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:6}

The following questions pertain to Lecture 20 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.
3. When studying time series modeling, we explored several ways to encode date information as a feature for the citibike dataset. When we used time of day as a numeric feature, the Ridge model was not able to capture the periodic pattern. Why? How did we tackle this problem? Briefly explain.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 6

1. Flood data would likely be unequally spaced because it does not uniformly occur necessarily at certain times of the year and it is random.

2. The lagged version would struggle more because they are made on the assumption that there is a uniform way that the data is provided to the model, so the lag would no longer represent the reality of the data and would not be able to communicate to the model that the uniformity has been disrupted. However, representing the date as "simple" data allows for the model to take into account the actual time between each instance.

3. This is because Ridge is a linear model and is not able to capture non-linear patterns (including cyclic), by encoding time as categorial variables, we were able to tackle it.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Computer vision 
rubric={points:6}

The following questions pertain to Lecture 19 on multiclass classification and introduction to computer vision. 

1. How many parameters (coefficients and intercepts) will `sklearn`’s `LogisticRegression()` model learn for a four-class classification problem, assuming that you have 10 features? Briefly explain your answer.
2. In Lecture 19, we briefly discussed how neural networks are sort of like `sklearn`'s pipelines, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?
3. Imagine that you have a small dataset with ~1000 images containing pictures and names of 50 different Computer Science faculty members from UBC. Your goal is to develop a reasonably accurate multi-class classification model for this task. Describe which model/technique you would use and briefly justify your choice in one to three sentences.

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 6

1. 44 total parameters, because for each class it learns 10 coefficients (one for each feat) and intercept for each class. so (10+1)*4 = 44

2. Because we can choose to take the preliminary and fundemental transformations (first steps of the "pipeline that make the data readible to the model") used by the pre-trained models and apply them to our own data and problems.

3. A pre-trained CNN model and fine-tuning it to my specific faculty data set. This is because it is very hard to make a model from scratch that is able to derive meaning from pictures and 1000 total pictures from a total of 50 people might not be enogh. Therefore, using a CNN that is trained with human faces and fine tuning that is the more efficient way to tackle the issue.

<!-- END QUESTION -->

<br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)